In [2]:
import time
import pymysql
import datetime

# 打开数据库连接
db = pymysql.connect("192.168.1.94", "dataUser94",
                     "94dataUser@2020", "saicnqms")

# 使用cursor()方法获取操作游标
cursor = db.cursor()

the_day_before_yesterday = time.strftime(
    "%Y-%m", time.localtime()) + '-' + str(int(time.strftime("%d", time.localtime())) - 3)
last_month = datetime.datetime.now().month - 1
last_month = f"0{str(last_month)}" if last_month < 10 else last_month
last_month = str(datetime.datetime.now().year) + '-' + str(last_month) + '-01'
print(the_day_before_yesterday, last_month)

2020-10-23 2020-09-01


In [3]:
# SQL 查询语句
# 总url
try:
#     获取表头
    cursor.execute('''select * from autohome_luntan_allurl;''')
    cols1 = [i[0] for i in cursor.description]
    print(cols1)
    # 执行SQL语句
    sql1 = "SELECT * FROM autohome_luntan_allurl WHERE posted_time <= '{}' AND  posted_time >= '{}'".format(
    the_day_before_yesterday, last_month)
    cursor.execute(sql1)
    # 获取所有记录列表
    results1 = cursor.fetchall()
    print(len(results1))
#     print(results1)
except:
    print("总URL表查询失败")

['grabtime', 'posted_time', 'car_id', 'user_car', 'page', 'brand', 'factory', 'tiezi_url', 'status']
109932


In [4]:
# 已经抓取的url
try:
    #     获取表头
    cursor.execute('''select * from luntan_all_copy;''')
    cols2 = [i[0] for i in cursor.description]
    print(cols2)
    # 执行SQL语句
    sql2 = "SELECT * FROM luntan_all_copy WHERE posted_time <= '{}' AND posted_time >= '{}' AND information_source = 'autohome'".format(
        the_day_before_yesterday, last_month)
    cursor.execute(sql2)
    results2 = cursor.fetchall()
    print(len(results2))
except:
    print("luntan_all_copy表查询失败")

# 关闭数据库连接

db.close()

['id', 'brand', 'click_num', 'content', 'content_num', 'factory', 'grabtime', 'information_source', 'parsetime', 'posted_time', 'province', 'region', 'reply_num', 'statusplus', 'title', 'url', 'user_car', 'user_name', 'create_time']
180180


In [6]:
import pandas as pd
import numpy as np

df1 = pd.DataFrame(np.array(results1), columns=cols1)
print('总url表去重前：', len(df1))
df1 = df1.drop_duplicates('tiezi_url')
print('总url表去重后：', len(df1))
df1['tiezi_url'] = df1['tiezi_url'].map(lambda x: x.replace('https', 'http'))

print(df1)

总url表去重前： 109932
总url表去重后： 109932
                   grabtime          posted_time car_id    user_car page  \
0       2020-10-26 10:33:13  2020-10-22 16:25:50   2531        都灵论坛    1   
1       2020-10-26 10:33:13  2020-10-20 19:20:26   2531        都灵论坛    1   
2       2020-10-26 10:33:13  2020-10-20 18:29:46   5057  依维柯褒迪新能源论坛    1   
3       2020-10-26 10:33:13  2020-10-18 22:45:41   5057  依维柯褒迪新能源论坛    1   
4       2020-10-26 10:33:13  2020-10-18 22:47:37   2531        都灵论坛    1   
...                     ...                  ...    ...         ...  ...   
109927  2020-10-26 12:25:03  2020-09-01 10:00:09    166     别克GL8论坛   19   
109928  2020-10-26 12:25:03  2020-09-01 09:54:56    166     别克GL8论坛   19   
109929  2020-10-26 12:25:03  2020-09-01 09:42:11    166     别克GL8论坛   19   
109930  2020-10-26 12:25:03  2020-09-01 09:14:22    166     别克GL8论坛   19   
109931  2020-10-26 12:25:03  2020-09-01 01:23:54    166     别克GL8论坛   19   

       brand factory                                 

In [7]:
df2 = pd.DataFrame(np.array(results2), columns=cols2)
print('已抓取表去重前：', len(df2))
df2 = df2.drop_duplicates('url')
print('已抓取表去重后：', len(df2))
df2['url'] = df2['url'].map(lambda x: x.replace('https', 'http'))

print(df2)

已抓取表去重前： 180180
已抓取表去重后： 94900
             id brand click_num  \
0             1    荣威      5030   
1             2    荣威        24   
2             3    荣威      5029   
3             5   雪佛兰       283   
4             6    荣威        20   
...         ...   ...       ...   
180170  2715653    荣威        11   
180172  2715707    荣威       121   
180174  2715807    荣威        53   
180175  2715868    荣威        60   
180178  2718995    荣威        48   

                                                  content content_num factory  \
0       我家的大白（荣威RX5 MAX）也是已经购买半年多了，当初买车时也是看了很多车，目标就是选定...          22   上汽乘用车   
1       哈哈，今天和媳妇一起去溜达，嘿嘿大家都在上班，我们却在玩，哈哈！因为我们两人的年休假还有没用...           6   上汽乘用车   
2       “一叶落知天下秋”，小小的落叶，如纤纤玉手，拨动了四季轮回中“秋”的琴弦，于是，清新轻盈的旋...          47   上汽乘用车   
3                 请教顶配车主，xl的毫米波雷达装在哪个位置呢？咋样的？有哪位兄弟有图片吗？谢谢          24    上汽通用   
4       随着荣威汽车不断的被大家所接受，荣威在产品的各个方面都做的非常考究，荣威RX5MAX的上市，...          10   上汽乘用车   
...                                                   ...     

In [8]:
df = df1.append(df2)
df = df.append(df2)
df.head()

,grabtime,posted_time,car_id,user_car,page,brand,factory,tiezi_url,status,id,...,information_source,parsetime,province,region,reply_num,statusplus,title,url,user_name,create_time
0,2020-10-26 10:33:13,2020-10-22 16:25:50,2531,都灵论坛,1,依维柯,南京依维柯,http://club.autohome.com.cn/bbs/thread/9aebc99...,http://club.autohome.com.cn/bbs/thread/9aebc99...,NaN,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-10-26 10:33:13,2020-10-20 19:20:26,2531,都灵论坛,1,依维柯,南京依维柯,http://club.autohome.com.cn/bbs/thread/69e44da...,http://club.autohome.com.cn/bbs/thread/69e44da...,NaN,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-10-26 10:33:13,2020-10-20 18:29:46,5057,依维柯褒迪新能源论坛,1,依维柯,南京依维柯,http://club.autohome.com.cn/bbs/thread/a6e02bd...,http://club.autohome.com.cn/bbs/thread/a6e02bd...,NaN,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-10-26 10:33:13,2020-10-18 22:45:41,5057,依维柯褒迪新能源论坛,1,依维柯,南京依维柯,http://club.autohome.com.cn/bbs/thread/eef3406...,http://club.autohome.com.cn/bbs/thread/eef3406...,NaN,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-10-26 10:33:13,2020-10-18 22:47:37,2531,都灵论坛,1,依维柯,南京依维柯,http://club.autohome.com.cn/bbs/thread/2ed45db...,http://club.autohome.com.cn/bbs/thread/2ed45db...,NaN,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df0 = df.drop_duplicates(subset=['tiezi_url', 'url'], keep=False)
df0.shape

(109932, 23)

In [10]:
df.shape

(299732, 23)

In [11]:
109932 + 94900 * 2

299732

In [22]:
print(df1.shape)
print(df2.shape)

(109932, 9)
(94900, 19)


In [50]:
ser1.compare(ser2)

AttributeError: 'Series' object has no attribute 'compare'